# RAG utilizând Azure AI Agent Service și Semantic Kernel

Acest fragment de cod demonstrează cum să creezi și să gestionezi un agent Azure AI pentru generarea augmentată prin recuperare (RAG) folosind `Azure AI Agent Service` și `Semantic Kernel`. Agentul procesează interogările utilizatorului pe baza contextului recuperat și oferă răspunsuri precise în consecință.


Corecție pentru versiunea SQLite  
Dacă întâmpinați eroarea:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Debifați acest bloc de cod la începutul notebook-ului:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importarea Pachetelor
Codul următor importă pachetele necesare:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Generare Augmentată prin Recuperare cu Semantic Kernel & Azure AI Agent Service

Acest exemplu demonstrează cum să utilizați **Azure AI Agent Service** pentru a realiza **Generare Augmentată prin Recuperare (RAG)** prin combinarea unui model lingvistic cu context specific domeniului dintr-un document încărcat.

### Cum Funcționează

1. **Încărcarea Documentului**: Un fișier markdown (document.md) care conține informații (politica de asigurare de călătorie a Contoso) este încărcat în serviciul agentului.

2. **Crearea Magazinului de Vectori**: Documentul este indexat într-un magazin de vectori pentru a permite căutarea semantică în conținutul său.

3. **Configurarea Agentului**: Un agent este instanțiat utilizând modelul `gpt-4o` cu următoarele instrucțiuni stricte:
   - Să răspundă doar la întrebări bazate pe conținutul recuperat din document.
   - Să refuze să răspundă dacă întrebarea este în afara domeniului de aplicare.

4. **Integrarea Instrumentului de Căutare în Fișiere**: `FileSearchTool` este înregistrat cu agentul, permițând modelului să caute și să recupereze fragmente relevante din documentul indexat în timpul inferenței.

5. **Interacțiunea cu Utilizatorul**: Utilizatorii pot pune întrebări. Dacă se găsesc informații relevante în document, agentul generează un răspuns fundamentat.  
   Dacă nu, agentul răspunde explicit că documentul nu conține informații suficiente.


### Funcția Principală

Aceasta este funcția principală a aplicației. Asigură-te că toate variabilele sunt inițializate corect înainte de a o apela.

```python
def main():
    # Inițializarea aplicației
    initialize_app()

    # Executarea funcției principale
    run_main_process()
```

### Configurare

Această secțiune explică cum să configurezi aplicația înainte de rulare.

1. Deschide fișierul de configurare.
2. Modifică valorile necesare.
3. Salvează modificările.

### Erori Comune

#### Eroare de inițializare
Această eroare apare atunci când aplicația nu este configurată corect. Verifică fișierul de configurare pentru a te asigura că toate valorile sunt setate corect.

#### Eroare de rulare
Această eroare apare dacă funcția principală întâmpină o problemă. Verifică logurile pentru mai multe detalii.

### Sfaturi

- Folosește funcția `log_error` pentru a înregistra erorile.
- Testează aplicația pe un mediu de dezvoltare înainte de a o lansa în producție.


Asigurați-vă că rulați mai întâi `az login` folosind Azure CLI pentru ca contextul de autentificare corespunzător să fie furnizat în timp ce utilizați `DefaultAzureCredential`. Serviciul Azure AI Agent nu utilizează chei API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Desigur! Te rog să îmi furnizezi fișierul Markdown pe care dorești să-l traduc.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original, în limba sa natală, ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de oameni. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
